In [ ]:
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os

# 1. Charger les variables d'environnement
load_dotenv()

# 2. Lire les variables depuis le fichier .env
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# 3. Connexion à la base de données
conn = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)

# 4. Requête SQL (tu peux retirer le LIMIT si tu veux tout)
query = """
SELECT id_article, title, abstract, year, doi,domain
FROM articles
WHERE abstract IS NOT NULL AND abstract != ''
"""

# 5. Chargement des résultats dans un DataFrame
df = pd.read_sql(query, conn)

# 6. Fermer la connexion
conn.close()


chargement du module 

In [ ]:
from sentence_transformers import SentenceTransformer

# Charger le modèle
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
# S'assurer que les abstracts sont en string
abstracts = df['abstract'].astype(str).tolist()

# Vectorisation avec barre de progression
embeddings = model.encode(abstracts, show_progress_bar=True, convert_to_numpy=True)


moteur de recherche faiss

In [ ]:
import faiss
import numpy as np

# Obtenir la dimension des vecteurs
dim = embeddings.shape[1]

# Créer l’index FAISS (utilise la distance euclidienne L2)
index = faiss.IndexFlatL2(dim)

# Ajouter les vecteurs à l’index
index.add(embeddings)

# ✅ Sauvegarder l’index
faiss.write_index(index, "faiss_index_articles.index")

# ✅ Sauvegarder les ID des articles associés aux embeddings (important pour la recherche plus tard)
np.save("indexed_article_ids.npy", df['id_article'].values)

# (Facultatif) Sauvegarder les métadonnées pour vérification humaine
df[['id_article', 'title', 'abstract', 'year', 'doi']].to_csv("articles_metadata.csv", index=False)


fonction de recherche

In [ ]:
import faiss
import numpy as np
import mysql.connector
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

# Charger l’index FAISS
index = faiss.read_index("faiss_index_articles.index")

# Charger les ID alignés avec les embeddings
article_ids = np.load("indexed_article_ids.npy",allow_pickle=True)

# Charger le modèle de recherche sémantique
model = SentenceTransformer("all-MiniLM-L6-v2")

# Fonction de recherche
def search_semantically(query, k=5):
    # Étape 1 : encoder la requête
    query_embedding = model.encode([query])

    # Étape 2 : rechercher les plus proches voisins dans FAISS
    distances, indices = index.search(query_embedding, k)

    # Étape 3 : récupérer les IDs d'article correspondants
    ids_articles = [article_ids[i] for i in indices[0]]


    if not ids_articles:
        return []

    # Étape 4 : se connecter à la base de données
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )
    cursor = conn.cursor(dictionary=True)

    # Construire la requête SQL dynamique
    placeholders = ','.join(['%s'] * len(ids_articles))
    sql = f"""
        SELECT a.id_article, a.title, a.abstract, a.year, a.doi,
               GROUP_CONCAT(au.author SEPARATOR ', ') AS authors
        FROM articles a
        LEFT JOIN article_author aa ON a.id_article = aa.id_article
        LEFT JOIN authors au ON aa.author_id = au.author_id
        WHERE a.id_article IN ({placeholders})
        GROUP BY a.id_article
    """
    cursor.execute(sql, ids_articles)
    results = cursor.fetchall()

    cursor.close()
    conn.close()

    return results
query = "medical diagnosis"
results = search_semantically(query, k=5)

if results:
    print(f"\nNombre de résultats trouvés : {len(results)}")
    for i, article in enumerate(results, 1):
        print(f"\n--- Résultat {i} ---")
        print(f"Titre     : {article['title']}")
        print(f"Auteur(s) : {article['authors']}")
        print(f"Année     : {article['year']}")
        print(f"DOI       : {article['doi']}")
        print(f"Résumé    : {article['abstract'][:300]}...")
else:
    print("Aucun résultat trouvé.")
